In [4]:
import pandas as pd
import numpy as np
import math
import warnings
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
warnings.filterwarnings('ignore')
np.random.seed(0)

## Group 7 - Kernel Implementation
### Author: Jasper De Landsheere

### Import data

In [8]:
df = pd.read_csv("diterpene_shuf.csv")
df

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a36,a37,a38,a39,a40,a41,a42,a43,a44,a45c
0,2,4,8,6,v1493,t,139.2,q,33.2,d,...,134.5,t,24.2,q,14.4,t,42.2,t,18.6,52c
1,2,4,8,6,v1255,d,144.0,q,66.7,d,...,22.4,s,37.7,q,19.7,q,16.5,t,26.6,54c
2,2,4,8,6,v2194,d,30.7,t,26.8,t,...,16.7,t,35.4,q,65.8,s,42.4,d,139.8,54c
3,3,4,9,4,v2021,t,37.2,t,17.4,q,...,33.5,d,44.9,d,56.6,q,20.8,t,36.5,3c
4,3,4,9,4,v1051,q,12.2,d,154.6,t,...,49.5,t,38.8,t,71.4,s,36.8,s,46.2,3c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,2,3,8,7,v501,q,18.8,t,32.5,d,...,172.1,q,62.3,d,36.1,t,27.0,s,38.8,54c
1198,2,4,8,6,v785,q,60.6,t,29.3,t,...,59.1,s,39.7,d,142.6,q,18.8,d,36.2,54c
1199,2,4,8,6,v2174,q,166.1,q,62.5,s,...,44.5,d,55.6,d,147.3,q,107.0,t,47.1,52c
1200,2,4,8,6,v1332,t,36.5,d,46.4,t,...,17.3,t,120.4,q,17.8,d,144.4,t,35.9,54c


# Data exploration and preprocessing

### Short explaination of the dataset

*Each diterpene is described with NMR spectroscopy. In this dataset each spectrum is described by the frequency and multiplicity of all peaks. The skeleton of every diterpene contains 20 carbon atoms. So, for every carbon atom we have its frequency and multiplicity. For multiplicity we have s(singulet), d(doublet), t(triplet), q(quartet). If an atom of carbon is connected to no hydrogen atoms its singulet, if its connected to one hydrogen its doublet, two hydrogens triplet and three hydrogens connected to one carbon atom is quartet.*

- Columns a0 to a3 represent how many times s, d, t, q values have occured.
- Column a4 is sample id.
- Columns a5 to a44 tells us the frequency and multiplicity of each carbon atom.
- Column a45c represents the classification class.

Example, for sample 0:
In a5 to a44, 's' value has occured 2 times, 'd' 4 times, 't' 8 times and 'q' 6 times. 'v1493' is its ID value. Then for the next 40 columns we have for each of the 20 carbon atoms its multiplicity (s,d,t,q) and frequency. Finally, in a45c we can see that sample 0 belongs to class 52c. 

--------------------
### Rename according exercise

In [9]:
# Drop first four columns as they are not needed according to the exercise
df = df.drop(["a0", "a1", "a2", "a3"], axis=1)
# Rename column "a45c" to "Class", "a4" to "ID"
df = df.rename(columns={"a45c": "Class", "a4": "ID"})
# Inspect
df.head()

,ID,a5,a6,a7,a8,a9,a10,a11,a12,a13,...,a36,a37,a38,a39,a40,a41,a42,a43,a44,Class
0,v1493,t,139.2,q,33.2,d,30.7,d,49.6,t,...,134.5,t,24.2,q,14.4,t,42.2,t,18.6,52c
1,v1255,d,144.0,q,66.7,d,46.2,t,32.7,q,...,22.4,s,37.7,q,19.7,q,16.5,t,26.6,54c
2,v2194,d,30.7,t,26.8,t,41.2,t,139.3,q,...,16.7,t,35.4,q,65.8,s,42.4,d,139.8,54c
3,v2021,t,37.2,t,17.4,q,16.7,s,58.1,t,...,33.5,d,44.9,d,56.6,q,20.8,t,36.5,3c
4,v1051,q,12.2,d,154.6,t,18.7,t,37.1,t,...,49.5,t,38.8,t,71.4,s,36.8,s,46.2,3c


In [10]:
# Create a copy
df_reduced = df.copy()

Based on the paper we can improve the results:

In [12]:
# For each sample of the dataset
col_names = df_reduced.columns
for i in tqdm(range(len(df_reduced))):
    
    for j in range (len(col_names)):

        if df_reduced[col_names[j]][i] == 's' and df_reduced[col_names[j+1]][i] >= 64.5 and df_reduced[col_names[j+1]][i] <= 95:
            df_reduced[col_names[j]][i] = 'd'

        if df_reduced[col_names[j]][i] == 's' and df_reduced[col_names[j+1]][i] >= 96 and df_reduced[col_names[j+1]][i] <= 114:
            df_reduced[col_names[j]][i] = 't'

        if df_reduced[col_names[j]][i] == 's' and df_reduced[col_names[j+1]][i] >= 115 and df_reduced[col_names[j+1]][i] <= 165:
            df_reduced[col_names[j]][i] = 'd'

        if df_reduced[col_names[j]][i] == 's' and df_reduced[col_names[j+1]][i] >= 165 and df_reduced[col_names[j+1]][i] <= 188:
            df_reduced[col_names[j]][i] = 'q'

        if df_reduced[col_names[j]][i] == 's' and df_reduced[col_names[j+1]][i] >= 188 and df_reduced[col_names[j+1]][i] <= math.inf:
            df_reduced[col_names[j]][i] = 't'


        if df_reduced[col_names[j]][i] == 'd' and df_reduced[col_names[j+1]][i] >= 64.5 and df_reduced[col_names[j+1]][i] <= 95:
            df_reduced[col_names[j]][i] = 't'

        if df_reduced[col_names[j]][i] == 'd' and df_reduced[col_names[j+1]][i] >= 105 and df_reduced[col_names[j+1]][i] <= 180:
            df_reduced[col_names[j]][i] = 't'

        if df_reduced[col_names[j]][i] == 'd' and df_reduced[col_names[j+1]][i] >= 96 and df_reduced[col_names[j+1]][i] <= 104:
            df_reduced[col_names[j]][i] = 'q'

        if df_reduced[col_names[j]][i] == 'd' and df_reduced[col_names[j+1]][i] >= 180 and df_reduced[col_names[j+1]][i] <= math.inf:
            df_reduced[col_names[j]][i] = 'q'


        if df_reduced[col_names[j]][i] == 't' and df_reduced[col_names[j+1]][i] >= 59 and df_reduced[col_names[j+1]][i] <= 90:
            df_reduced[col_names[j]][i] = 'q'

        if df_reduced[col_names[j]][i] == 't' and df_reduced[col_names[j+1]][i] >= 90 and df_reduced[col_names[j+1]][i] <= math.inf:
            df_reduced[col_names[j]][i] = 'q'

100%|██████████| 1202/1202 [00:02<00:00, 590.50it/s]


In [13]:
df_reduced.head()

,ID,a5,a6,a7,a8,a9,a10,a11,a12,a13,...,a36,a37,a38,a39,a40,a41,a42,a43,a44,Class
0,v1493,q,139.2,q,33.2,d,30.7,d,49.6,t,...,134.5,t,24.2,q,14.4,t,42.2,t,18.6,52c
1,v1255,q,144.0,q,66.7,d,46.2,t,32.7,q,...,22.4,s,37.7,q,19.7,q,16.5,t,26.6,54c
2,v2194,d,30.7,t,26.8,t,41.2,q,139.3,q,...,16.7,t,35.4,q,65.8,s,42.4,q,139.8,54c
3,v2021,t,37.2,t,17.4,q,16.7,s,58.1,t,...,33.5,d,44.9,d,56.6,q,20.8,t,36.5,3c
4,v1051,q,12.2,q,154.6,t,18.7,t,37.1,q,...,49.5,t,38.8,q,71.4,s,36.8,s,46.2,3c


In [14]:
# Compute RBF kernel
def rbf_kernel(x, y, gamma):
    return np.exp(-gamma * np.linalg.norm(x - y) ** 2)

# Compute Gram matrix
def gram_matrix(X, gamma):
    n_samples, n_features = X.shape
    K = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(i+1):
            kernel_sum = 0
            for element_i in range(n_features):
                for element_j in range(n_features):
                    # If the elements are the same and they are strings
                    if X[i][element_i] == X[j][element_j] and type(X[i][element_i]) == str:
                        kernel_sum += rbf_kernel(X[i][element_i + 1], X[j][element_j +1 ], gamma) 
            K[i, j] = kernel_sum
            K[j, i] = kernel_sum
    return K

def test_gram_matrix(X_train, X_test, gamma):
    n_train = X_train.shape[0]
    n_test = X_test.shape[0]
    n_features=X_test.shape[1]
    K = np.zeros((n_test, n_train))
    for i in range(n_test):
        for j in range(n_train):
            kernel_sum = 0
            for element_i in range(n_features):
                for element_j in range(n_features):
                    if X_test[i][element_i] == X_train[j][element_j] and type(X_test[i][element_i]) == str:
                        kernel_sum += rbf_kernel(X_test[i][element_i + 1], X_train[j][element_j + 1], gamma)
            K[i, j] = kernel_sum
    return K

In [16]:
# Split data into training and test sets
X = df_reduced.drop(["Class", "ID"], axis=1)
y = df_reduced["Class"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Compute Gram matrix for training set
gamma = 0.06
K_train = gram_matrix(X_train.to_numpy(), gamma=gamma)

In [17]:
# Dimension K_train
K_train.shape, X_train.shape

((961, 961), (961, 40))

In [18]:
# Check if K_train is symmetric
np.allclose(K_train, K_train.T)

True

In [19]:
# Compute Gram matrix for test set
K_test = test_gram_matrix(X_train.to_numpy(), X_test.to_numpy(), gamma=gamma)

In [20]:
# Dimension K_test
K_test.shape, X_test.shape

((241, 961), (241, 40))

In [21]:
# Check if Gram matrix is symmetric
np.allclose(K_train, K_train.T, atol=1e-8)

True

In [22]:
# import svc
from sklearn.svm import SVC

# Train SVM
clf = SVC(kernel='precomputed', C=1)
clf.fit(K_train, y_train)

SVC(C=1, kernel='precomputed')

In [23]:
# Make predictions
y_pred = clf.predict(K_test)

In [24]:
# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.946058091286307
